In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [21]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data import make_dataset
from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax

ModuleNotFoundError: No module named 'pptx'

## Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s


In [3]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [5]:
display(metricas.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2014,107100,"""Hospital Dr. G…","""N390""",306,2515,9,23
2008,111200,"""Hospital Clíni…","""M713""",12,13,11,0
2001,129106,"""Hospital San J…","""I832""",1,18,0,0
2014,115206,"""Hospital Clíni…","""M179""",34,181,29,0
2008,115103,"""Hospital Del S…","""F102""",10,35,2,0
2015,124250,"""Clínica Puerto…","""I829""",1,1,1,0
2020,116100,"""Hospital San J…","""O998""",2,3,2,0
2001,123200,"""Clínica Aleman…","""O244""",1,2,0,0
2020,112200,"""Clínica Aleman…","""L720""",11,11,11,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
  estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
  hospitales - diagnosticos distintos.


Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
  cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
  El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
  etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)


## Análisis de Estratos


In [6]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_A_ANLIZAR)

In [7]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)

In [8]:
diags_mas_relevantes = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_19856\516069765.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)


In [46]:
DIAGS_MAS_RELEVANTES = veinte_mas_relevantes_por_anio.select(pl.col("DIAG1").unique()).sort(
    by="DIAG1"
)

In [52]:
print(DIAGS_MAS_RELEVANTES)

shape: (46, 1)
┌───────┐
│ DIAG1 │
│ ---   │
│ str   │
╞═══════╡
│ C33X  │
│ C340  │
│ C341  │
│ C342  │
│ …     │
│ Q676  │
│ T820  │
│ T821  │
│ Z450  │
└───────┘


In [53]:
DIAGS_1_o_2_en_2019 = (
    rankings_hospital_analizado.filter(
        (pl.col("DIAG1").is_in(DIAGS_MAS_RELEVANTES.to_series()))
        & (pl.col("ANO_EGRESO") == 2019)
        & (
            (pl.col("ranking_nacionales_n_egresos") == 1)
            | (pl.col("ranking_nacionales_n_egresos") == 2)
        )
    )
    .select(pl.col("DIAG1"))
)

In [54]:
print(DIAGS_1_o_2_en_2019)

shape: (35, 1)
┌───────┐
│ DIAG1 │
│ ---   │
│ str   │
╞═══════╡
│ Z450  │
│ T821  │
│ T820  │
│ M348  │
│ …     │
│ C342  │
│ C341  │
│ C340  │
│ C33X  │
└───────┘


Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos


In [11]:
display(rankings_nacionales.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2008,117101,"""Hospital Clíni…","""Cap.18 SÍNTOM…","""R50-R69 SÍNTO…","""R59 ADENOMEGAL…","""Adenomegalia g…","""R591""",1,8,0,0,13,26,0.038462,8,14,0.071429,null,null,null,7,10,0.1,null,null,null
2017,112528,"""Clínica Las Li…","""Cap.17 MALFOR…","""Q38-Q45 OTRAS…","""Q41 AUSENCIA, …","""Ausencia, atre…","""Q412""",1,11,1,0,6,13,0.076923,null,null,null,3,6,0.166667,null,null,null,null,null,null
2005,121119,"""Hospital de Go…","""Cap.04 ENFERM…","""E08-E14 DIABE…","""E14 DIABETES M…","""Diabetes melli…","""E146""",8,22,0,0,16,616,0.012987,15,563,0.01421,null,null,null,null,null,null,null,null,null
2017,114105,"""Hospital Clíni…","""Cap.10 ENFERM…","""J00-J06 INFEC…","""J04 LARINGITIS…","""Traqueítis agu…","""J041""",1,2,0,0,15,59,0.016949,11,41,0.02439,null,null,null,10,34,0.029412,null,null,null
2005,122201,"""Hospital Padre…","""Cap.09 ENFERM…","""I80-I89 ENFER…","""I88 LINFADENIT…","""Linfadenitis m…","""I880""",14,31,0,0,13,688,0.020349,7,408,0.034314,null,null,null,null,null,null,null,null,null
2018,107217,"""Hospital Naval…","""Cap.11 ENFERM…","""K00-K14 ENFER…","""K11 ENFERMEDAD…","""Sialolitiasis""","""K115""",2,2,2,0,20,108,0.018519,null,null,null,14,66,0.030303,null,null,null,null,null,null
2018,121202,"""Clínica Aleman…","""Cap.21 FACTOR…","""Z40-Z54 CONTA…","""Z47 OTROS CUID…","""Otros cuidados…","""Z478""",1,2,1,0,34,202,0.00495,null,null,null,14,122,0.008197,null,null,null,null,null,null
2019,110270,"""Clínica Los Ma…","""Cap.09 ENFERM…","""I60-I69 ENFER…","""I67 OTRAS ENFE…","""Enfermedad cer…","""I679""",1,8,0,0,124,1199,0.000834,null,null,null,38,299,0.003344,null,null,null,null,null,null
2019,109201,"""Clínica Dávila…","""Cap.13 ENFERM…","""M20-M25 OTROS…","""M21 OTRAS DEFO…","""Otras deformid…","""M215""",4,5,4,0,20,323,0.012384,null,null,null,11,206,0.019417,null,null,null,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.


In [12]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.


In [13]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [14]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()

In [15]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (pl.col("ANO_EGRESO") == 2019)
)

In [16]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## 3. Análisis de procedimientos DEIS

En este análisis se quieren responder las siguientes preguntas:

1. ¿Cuál es el procedimiento más frecuente dentro de los pacientes que se les realizó un procedimiento
   quirúrgico?


In [17]:
proced = (
    df_nacional.filter(pl.col("INTERV_Q") == 1)
    .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])
    .agg(pl.count())
).collect()

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_19856\3397463805.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])


In [18]:
display(proced)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_PROCED_PPAL,count
i64,i64,str,u32
2001,103203,null,5921
2001,107212,null,1504
2001,106100,null,9333
2001,107100,null,9959
2001,121111,null,309
2001,105103,null,262
2001,117101,null,1079
2001,112239,null,5848
2001,103210,null,1699


Los resultados muestran el desglose por año y recinto hospitalario junto a la cantidad de
procedimientos realizados en cada intervención quirúrgica. Sin embargo, se puede observar que
existe una cantidad importante de casos donde la glosa del procedimiento fue nula.

Ahora, se quiere calcular cuántas intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [19]:
display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_19856\989951252.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))


GLOSA_PROCED_PPAL,count
str,u32
"""Enema baritado…",1
"""tonometria ocu…",1
"""Histerosalping…",1
"""Cócleovestibul…",1
"""Criocoagulacio…",1
"""Radiografía Pa…",1
"""Resonancia Mag…",1
"""Tratamiento po…",1
"""Electrocardiog…",1


Por lo tanto, 12382642 de todas las intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [20]:
proced.write_excel("../data/interim/conteo_procedimientos.xlsx")